## **View MitoNet segmentations in 3D**
---
Visualize results from MitoNet segmentation

|   | Dataset                   | Figure | Dwell time (µs) | No. of sections | ROA size (µm) |
|---|---------------------------|--------|-----------------|-----------------|---------------|
|   | 20230711_MCF7_UAc         | 6      | 20              | 54              | 240 x 240     |
|   | 20230626_Rat_Pancreas_OTO | S3, S5 | 10              | 44              | 96 x 96       |

In [1]:
# Data import
import webknossos as wk
import numpy as np

# Napari
import napari
import src.importo

In [2]:
# Create an empty viewer
viewer = napari.Viewer()

### **3.1 Configure dataset parameters**
---

#### **Dataset name**
---
Uncomment the one to view

In [3]:
# dataset_full_url = 'https://webknossos.tnw.tudelft.nl/links/5Q_PJXjPPa4P3XBO' # 20230626_Rat_Pancreas_OTO
dataset_full_url = 'https://webknossos.tnw.tudelft.nl/links/_eSz0uGGSzkZ1w__' # 20230711_MCF7_UAc

## Mag 
mag_x_seg, mag_y_seg, mag_z_seg = 4, 4, 1 # For MitoNet predictions 4, 4, 1 or larger, no 2, 2, 1 and 1, 1, 1 exist!
mag_x_EM, mag_y_EM, mag_z_EM = 4, 4, 1 # Show data at 4, 4, 1 Mag (4x downsampled / 16nm/pixel)

MAG_seg = wk.Mag(f"{mag_x_seg}-{mag_y_seg}-{mag_z_seg}")
MAG_EM = wk.Mag(f"{mag_x_EM}-{mag_y_EM}-{mag_z_EM}")

#### **Layers and bounding boxes**
---
- Uncomment under the dataset header the layers to analyze. Either select "fine aligned data" or "SOFIMA aligned data" block 
- Bounding boxes apply to specific views of the data with ground truth annotations. `20230626_Rat_Pancreas_OTO`: use either bounding box 1 or 2, for `20230711_MCF7_UAc` there is only one bounding box

`20230626_Rat_Pancreas_OTO`

In [4]:
## FINE ALIGNED DATA
# EM_layer = 'EM'
# GT_layer = 'Mito_GT_EM'
# mitonet_layer = 'EM_MitoNet'

## SOFIMA ALIGNED DATA
# EM_layer = 'EM_realigned_SOFIMA'
# GT_layer = 'Mito_GT_EM_realigned_SOFIMA'
# mitonet_layer = 'EM_realigned_SOFIMA_MitoNet'

In [5]:
# ## BBOX 1 (RAT PANCREAS)
# bbox_EM = wk.BoundingBox((14868, 13908, 0),
#                           (2141, 2210, 38)).align_with_mag(MAG_seg) # ((x0, y0, z0), (x_size, y_size, z_size))
# bbox_seg = wk.BoundingBox((14868, 13908, 0),
#                           (2141, 2210, 38)).align_with_mag(MAG_seg) # ((x0, y0, z0), (x_size, y_size, z_size))

## BBOX 2 (RAT PANCREAS)
# bbox_EM = wk.BoundingBox((17277, 21620, 0),
#                           (2141, 1963, 43)).align_with_mag(MAG_seg) # ((x0, y0, z0), (x_size, y_size, z_size))
# bbox_seg = wk.BoundingBox((17277, 21620, 0),
#                           (2141, 1963, 43)).align_with_mag(MAG_seg) # ((x0, y0, z0), (x_size, y_size, z_size))

`20230711_MCF7_UAc`

In [6]:
## FINE ALIGNED DATA
EM_layer = 'EM'
GT_layer = 'Mito_GT_EM'
mitonet_layer = 'GT_EM_MitoNet'

## SOFIMA ALIGNED DATA
# EM_layer = 'GT_EM_realigned_SOFIMA'
# GT_layer = 'Mito_GT_EM_realigned_SOFIMA'
# mitonet_layer = 'GT_EM_realigned_SOFIMA_MitoNet'

In [7]:
# BBOX 1 (MCF-7 cells)
bbox_EM = wk.BoundingBox((25184, 52192, 0),
                         (4416, 8256, 54)).align_with_mag(MAG_EM) # ((x0, y0, z0), (x_size, y_size, z_size))
bbox_seg = wk.BoundingBox((25184, 52192, 0),
                          (4416, 8256, 54)).align_with_mag(MAG_seg) # ((x0, y0, z0), (x_size, y_size, z_size))

### **3.2 Read data from WebKnossos (remote)**
---
Reads data, segmentations, and ground truth annotations from remote `WebKnossos` dataset

In [8]:
# For local access
dataset, voxelsize = src.importo.import_wk_dataset_remote(dataset_full_url=dataset_full_url)
scale_factor = voxelsize[2] / voxelsize[0] / (mag_x_EM / mag_z_EM) # determine scalefactor for viewing

In [9]:
EM = dataset.get_layer(EM_layer) # EM Layer
seg = dataset.get_layer(mitonet_layer) # MitoNet layer
gt = dataset.get_layer(GT_layer) # GT layer

mag_view = EM.get_mag(MAG_EM) # MagView
mag_view_seg = seg.get_mag(MAG_seg) # MagView
mag_view_gt = gt.get_mag(MAG_seg) # MagView

view = mag_view.get_view(absolute_offset=bbox_EM.topleft, size=bbox_EM.size) # "absolute_offset" and "size" are in Mag(1)!
view_seg = mag_view_seg.get_view(absolute_offset=bbox_seg.topleft, size=bbox_seg.size) # "absolute_offset" and "size" are in Mag(1)!
view_gt = mag_view_gt.get_view(absolute_offset=bbox_seg.topleft, size=bbox_seg.size) # "absolute_offset" and "size" are in Mag(1)!
    
data = view.read()
seg = view_seg.read()
gt = view_gt.read()

In [10]:
data, seg, gt = np.squeeze(data), np.squeeze(seg), np.squeeze(gt)
print(data.shape) # Coordinates are (x, y, z), but Napari weirdly reads this as (y, z, x)...
print(seg.shape) # Coordinates are (x, y, z), but Napari weirdly reads this as (z, x, y)...
print(gt.shape)# Coordinates are (x, y, z), but Napari weirdly reads this as (z, x, y)...

(1104, 2064, 54)
(1104, 2064, 54)
(1104, 2064, 54)


In [11]:
# Pick segmentations where GT exists for evaluation
seg_mitonet_filtered = np.where(gt > 0, seg, 0)

# Include also predicted pixels of objects outside of GT pixels
object_ids = np.unique(seg_mitonet_filtered)
objects = np.isin(seg, list(object_ids))
seg_mitonet_only_GT_objects = np.where(objects, seg, 0)

# Rename
seg = seg_mitonet_only_GT_objects

### **3.2 Add data + segmentations to `Napari` viewer**
---
Reads data and segmentations from WebKnossos into `Napari`

In [12]:
# Add data tp viewer
viewer.add_image(np.transpose(data, (2, 0, 1)),
                              scale=(scale_factor, 1, 1)) # Use scalefactor for realistic rendering

# Add segmentation
seg_layer = viewer.add_labels(np.transpose(seg, (2, 0, 1)), 
                              name='segmentation', 
                              scale=(scale_factor, mag_x_seg/mag_x_EM, mag_x_seg/mag_x_EM)) # Scale to EM  

# Add ground truth
gt_layer = viewer.add_labels(np.transpose(gt, (2, 0, 1)), 
                             name='ground truth', 
                             scale=(scale_factor, mag_x_seg/mag_x_EM, mag_x_seg/mag_x_EM)) # Scale to EM  